In [1]:
import torch
import random
import unidecode
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker


In [3]:
ALPHABET = 'abcdefghijklmnopqrstuvwxyz-'

def normalize(s):
    global ALPHABET
    s = unidecode.unidecode(s)
    s = s.lower()
    s = ''.join(c for c in s if c in ALPHABET)
    return s

normalize('HéRésIe!')

'heresie'